# Creator

TODO : handle punctuation

end good translation (v -> b)



In [46]:
import json, epitran
from nltk import ngrams

In [2]:
save_file = 'katakana2letter.json'
with open (save_file, encoding='utf8') as f :
    translator2letters = json.load(f)
epi = epitran.Epitran('fra-Latn')

In [178]:
phon_translator2letters = {
    tuple(epi.transliterate(let.replace('sh', 'ch'))): jap_let 
    for jap_let, let in translator2letters.items()
}

In [177]:
tuple(['a'])

('a',)

In [247]:
#phon_translator2letters

In [237]:
test = '''§ 276. Pour la nouvelle année. - Je vis encore, je pense encore : je dois vivre encore, car je dois encore penser. Sum, ergo cogito : cogito : ergo sum. Aujourd’hui, chacun ose exprimer son vœu et sa pensée la plus chère : soit ! Je veux donc dire moi aussi ce qu'aujourd'hui je me souhaitais à moi-même et quelle pensée a cette année été la première à traverser mon cœur – quelle pensée doit être le fondement, la garantie et la douceur de toute pensée à venir ! Je veux toujours plus apprendre à voir la nécessité dans les choses comme le beau – ainsi serai-je l'un de ceux qui rendent belles les choses. Amor fati : que cela soit à présent mon amour ! Je ne veux mener aucune guerre contre le laid. Je ne veux pas accuser, je ne veux pas même accuser les accusateurs. Que détourner le regard soit mon unique négation ! Et, en tout et pour tout, et en grand : je veux, en n'importe quelle circonstance, n'être rien d'autre que quelqu'un qui dit oui. '''

In [245]:
change_phon = {
    'ɛ': 'ə',
    'e': 'ə',
    'ɔ̃': 'on',
    'ɔ': 'on',
    'ɑ̃': 'an',
    'ɑ': 'an',
    '̃ ': '',
    '̃': '',
    'l': 'ʀ',
}
def phenotic_fr(s):
    s = s.replace('er', 'é').replace('ez', 'é')
    phons = epi.transliterate(s)
    if phons[-1] == s[-1] and phons[-1] in 'dpqstr' :
        phons = phons[:-1]
    phons = ''.join([change_phon.get(let, let) for let in phons if change_phon.get(let, let)])
    return phons
def phenotic_fr2jap_let(phons) :
    retake = 0
    res = ''
    for let in ngrams(phons, 3 , pad_right=True):
        
        if let[0] is None :
            break
        if retake > 0 :
            retake -= 1
        else :
            #print(res, let, phons)
            found_smthg = False
            for test_let, size_let in [
                (let, 3),
                (let[:2], 2),
                (let[:2] + ('y',), 2),
                (let[:1], 1),
                (let[:1] + ('y',), 1),
                (let[:1] + ('ə',), 1),
            ]:
                if test_let in phon_translator2letters :
                    res += phon_translator2letters[test_let]
                    retake = size_let - 1
                    found_smthg =  True
                    break
            if not found_smthg and let[0] == 'ʃ':
                res += phon_translator2letters[tuple('ʃi')]
                found_smthg =  True

            if not found_smthg:
                if not let in {
                    ('t', None, None),
                } :
                    print('problem', retake, let, '/', phons)

    return res if res else phons

def fr2katakana(text):
    phons = phenotic_fr(text)
    return ' '.join([phenotic_fr2jap_let(word) for word in phons.split(' ')])

In [248]:
#list(zip(fr2katakana(test).split(' '), test.split(' '), phenotic_fr(test).split(' ')))